In [ ]:
#Import libraries
import os
import pydicom
import numpy as np
import keras
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
#Import own utilities
import preproc
import net3d_model as model_3d
import utils
import keraOverlay as overlay

In [4]:
# Load data and masks as dicom files
data_dir = '../../data'                   #Please change the path accordingly before make any changes
normalNames = data_dir + '/GoodData_256auto.txt'
img_dir = data_dir + '/256Auto/images'
msk_dir = data_dir + '/256Auto/masks'

In [ ]:
n_slices = 128

model, prediction = model_3d.conv_net(size=128, activation_type="sigmoid", n_slices=n_slices, depth=5, n_base_filters=16)
#model.summary()
# This visualization might not work depending on how graphviz is installed, but it gives a simple overview of the structure.
#To do: output this image to file or in pop up window
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
#SVG(model_to_dot(model).create(prog='dot', format='svg'))
model.compile(loss=model_3d.dice_coef_loss, optimizer='adam', metrics=['acc', model_3d.dice_coef])


In [7]:
normal_namelist = []
with open(normalNames) as f:
    content = f.readlines()
    normal_namelist = [x.strip() for x in content] 


0 / 10 fold


Epoch 1/5


(1, 128, 128, 128)


(1, 128, 128, 128)


(1, 128, 128, 128)


...

(1, 128, 128, 128)


 2/15 [===>..........................] - ETA: 18:08 - loss: 0.9979 - acc: 0.2647 - dice_coef: 0.0021

(1, 128, 128, 128)


 3/15 [=====>........................] - ETA: 16:58 - loss: 0.9970 - acc: 0.3720 - dice_coef: 0.0030

(1, 128, 128, 128)


In [ ]:
#Cross Validation
k = 10   #Number of folds
n_batch = 1 #Number of batches

folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(np.array(normal_namelist), np.zeros(len(normal_namelist))))

for index in range(k):
    print(index+1, "/", k, "fold")
    train_list = [normal_namelist[i] for i in folds[index][0]]
    val_list = [normal_namelist[i] for i in folds[index][1]]
    #print(train_list, val_list)
    
    history = model.fit_generator(overlay.generate_batch_data(img_dir, msk_dir, train_list, batch_size=n_batch), 
                                  epochs=5, verbose = 1, steps_per_epoch=len(train_list)/n_batch,
                                  validation_data=overlay.generate_batch_data(img_dir,msk_dir, val_list, batch_size=1), validation_steps=len(val_list))


In [1]:
import matplotlib.pyplot as plt


In [ ]:
plt.title('Training Progression')
plt.plot(history.epoch, history.history['loss'], label='loss')
plt.plot(history.epoch, history.history['acc'], label='acc')

#plt.plot(history.epoch, history.history[model_3d.dice_coef], label="dice coef")  #TBD
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
n_slice = 64
size = 64

index = 2



#data = utils.padImage(x_list[index])
#data_tobox = preproc.to3dpatches(data, )
predictions = model.predict(x_list[index].reshape(1,n_slice,size,size,1), steps=1)

In [ ]:
slice_index = 34
#Show original, mask, prediction#Show or 
show = plt.figure()
data = show.add_subplot(1,3,1)
data.set_title("data")
#plt.imshow(x_list[0][66])
plt.imshow(x_list[index][slice_index].reshape(size,size))

label = show.add_subplot(1,3,2)
label.set_title("label")
plt.imshow(y_list[index][slice_index].reshape(size,size))

prediction = show.add_subplot(1,3,3)
prediction.set_title("prediction")
plt.imshow(predictions[0][slice_index].reshape(size,size))



In [ ]:
from keras import backend

print(predictions[0][slice_index].shape)

#dice = model_3d.dice_coef(predictions[0],x_list[index])
dice = model_3d.dice_coef(predictions[0],predictions[0])

print(predictions[0][slice_index].dtype)
#dice = dice_coef(y_list[index],predictions[0])
print(dice)
#dice = backend.print_tensor(dice*8, message='y_pred = ')
#backend.print_tensor(dice, message='')
#print(dice)
piece = predictions[0]
piece = piece[piece > 0.9]
dice = model_3d.dice_coef(piece,piece)

backend.eval(dice)
